In [23]:
import numpy as np
import random
import pandas as pd
import math

In [15]:
def _readfile(file):

    # ----- read data -----
    df = pd.read_csv(file)
    df = df.fillna(method = 'ffill')

    # ----- Create Features -----
    X = df.drop(['id','class'], axis=1).copy(deep=False)
    X = (X-X.min())/(X.max()-X.min())

    # ----- Create Desired outputs -----
    Y = df[['class']].copy(deep=False)

    Input = X.to_numpy()
    Output = Y.to_numpy()

    return Input,Output

Input,Output = _readfile('data/wdbc.csv')

In [17]:
def cross_validations_split(shape,folds):
    fold_size = int(shape * folds/100)
    k = 0
    index = []
    for i in range(1,folds+1):
        if i < folds:
            index.append([k,i*fold_size])
        else:
            index.append([k,shape])
        k = i*fold_size
    return index

In [ ]:
class GA(object):
    def __init__(self, hiddenSize, inputSize, outputSize):

        # initiate layers
        self.inputSize = inputSize
        self.outputSize = outputSize
        self.hiddenSize = hiddenSize

        layers = [self.inputSize] + self.hiddenSize + [self.outputSize]

        # initiate positions
        positions = []
        for i in range(len(layers)-1):
            p = np.random.uniform(-10,10,(layers[i], layers[i+1]))
            positions.append(p)
        self.positions = positions
        self.positions_best = positions

        velocitys = []
        for i in range(len(layers) - 1):
            v = np.random.uniform(-10,10,(layers[i], layers[i+1]))
            velocitys.append(v)
        self.velocitys = velocitys

    def feedForward(self, X):
        Output_node = X
        for i, p in enumerate(self.positions):
        
            v = np.dot(Output_node, p)
            Output_node = self.sigmoid(v)

        return Output_node

    def sigmoid(self, s, deriv=False):
        if (deriv == True):
            return s * (1-s)
        return 1/(1 + np.exp(-s))

    def object_funct(self, X, Y):

        seed = randint(1, 25*100)

        np.random.seed(seed)
        np.random.shuffle(X)

        np.random.seed(seed)
        np.random.shuffle(Y)

        sum_err = 0
        for j, input in enumerate(X):
            target = Y[j]
            output = self.feedForward(input)

            sum_err += self._mae(target, output)

        self.fx = sum_err/len(X) 
        return self.fx

    def _mse(self, target, output):
        return np.average(abs(target - output)**2)
